# 📊 Notebook 4: Account Monitoring (Functions Version)

## วัตถุประสงค์
ตรวจสอบ Account แบบ Real-time ด้วย **Functions + main()**

## ✨ Structure
- แต่ละ cell มี **function definition** + **main()** ของตัวเอง
- รันแต่ละ cell แยกอิสระ
- ดู Balance, Orders, Positions แบบ Real-time

---

## ⚙️ Cell 0: Setup (รันก่อนเสมอ)

In [9]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import display, HTML, clear_output
from pprint import pprint
import time
from datetime import datetime

from grvt_helpers import (
    load_grvt_config,
    connect_to_grvt,
    format_currency,
    format_percentage,
    create_account_table,
    create_positions_table,
    print_section_header
)

# Helper function
def safe_float(value, default=0.0):
    """แปลงค่าเป็น float อย่างปลอดภัย"""
    if value is None or value == 'N/A' or value == '':
        return default
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

# ตั้งค่า pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# โหลด config และเชื่อมต่อ
config = load_grvt_config()
client = connect_to_grvt(config)

if not client:
    print("❌ ไม่สามารถเชื่อมต่อกับ GRVT")
else:
    print(f"✅ พร้อมใช้งาน! (Environment: {config['GRVT_ENV']})")

✅ เชื่อมต่อกับ GRVT testnet สำเร็จ!
✅ พร้อมใช้งาน! (Environment: testnet)


---

## 💰 Cell 1: ดู Account Balance

In [10]:
# ==========================================
# Function Definition
# ==========================================

def get_account_balance(client):
    """
    ดึงข้อมูล Account Balance
    
    Args:
        client: GRVT client
    
    Returns:
        balance: Balance data หรือ None
    """
    print_section_header("💰 Account Balance")
    
    try:
        # ดึง balance
        balance = client.fetch_balance()
        
        # Extract key metrics
        total_equity = safe_float(balance.get('total', {}).get('USDT', 0))
        available = safe_float(balance.get('free', {}).get('USDT', 0))
        used_margin = safe_float(balance.get('used', {}).get('USDT', 0))
        
        # Display
        print(f"💎 Total Equity:       {format_currency(total_equity)}")
        print(f"✅ Available Balance:  {format_currency(available)}")
        print(f"📊 Used Margin:        {format_currency(used_margin)}")
        
        if total_equity > 0:
            margin_ratio = (used_margin / total_equity) * 100
            print(f"📈 Margin Utilization: {margin_ratio:.2f}%")
        
        print("\n✅ สำเร็จ!")
        return balance
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None


def display_balance_table(balance):
    """
    แสดง Balance ในรูปแบบตาราง
    
    Args:
        balance: Balance data
    """
    # สร้างตาราง
    account_df = create_account_table(balance)
    
    # จัดรูปแบบและแสดงผล
    print("\n📋 Account Summary Table:\n")
    display(account_df.style
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([{
            'selector': 'th',
            'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]
        }])
    )


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: ดู Account Balance"""
    balance = get_account_balance(client)
    if balance:
        display_balance_table(balance)
    return balance


# เรียกใช้
balance = main()


  💰 Account Balance

💎 Total Equity:       $198,100.33
✅ Available Balance:  $198,097.01
📊 Used Margin:        $3.32
📈 Margin Utilization: 0.00%

✅ สำเร็จ!

📋 Account Summary Table:



,Metric,Value
0,Total Equity,"$198,100.33"
1,Available Balance,"$198,097.01"
2,Used Margin,$3.32
3,Unrealized P&L,$0.00


---

## 📝 Cell 2: ดู Open Orders

In [11]:
# ==========================================
# Function Definition
# ==========================================

def get_open_orders(client):
    """
    ดึงและแสดง Open Orders
    
    Args:
        client: GRVT client
    
    Returns:
        open_orders: List of orders
    """
    print_section_header("📝 Open Orders")
    
    try:
        # ดึง open orders
        open_orders = client.fetch_open_orders()
        
        if not open_orders:
            print("ℹ️ ไม่มี open orders")
            print("💡 ลองส่ง limit order ใน Notebook 5 แล้วกลับมาเช็คที่นี่")
            return open_orders
        
        print(f"📊 มี {len(open_orders)} open order(s):\n")
        
        for idx, order in enumerate(open_orders, 1):
            # Parse GRVT order structure
            order_id = order.get('order_id', 'N/A')
            
            # Get first leg
            legs = order.get('legs', [])
            if legs:
                leg = legs[0]
                symbol = leg.get('instrument', 'N/A')
                amount = safe_float(leg.get('size', 0))
                price = safe_float(leg.get('limit_price', 0))
                is_buying = leg.get('is_buying_asset', True)
                side = 'BUY' if is_buying else 'SELL'
            else:
                symbol = 'N/A'
                amount = 0
                price = 0
                side = 'N/A'
            
            # Get state info
            state = order.get('state', {})
            status = state.get('status', 'N/A')
            
            traded_sizes = state.get('traded_size', ['0.0'])
            filled = safe_float(traded_sizes[0] if traded_sizes else 0)
            
            # Order type
            order_type = order.get('time_in_force', 'LIMIT')
            
            # Emoji
            side_emoji = "🟢" if side == "BUY" else "🔴" if side == "SELL" else "⚪"
            
            print(f"{idx}. {side_emoji} {symbol} - {side} {order_type}")
            print(f"   Order ID:  {str(order_id)[:40]}...")
            print(f"   Amount:    {amount:.4f}")
            print(f"   Price:     {format_currency(price)}")
            print(f"   Filled:    {filled:.4f} / {amount:.4f} ({(filled/amount*100) if amount > 0 else 0:.1f}%)")
            print(f"   Status:    {status}")
            print()
        
        return open_orders
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return []


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: ดู Open Orders"""
    orders = get_open_orders(client)
    return orders


# เรียกใช้
orders = main()


  📝 Open Orders

ℹ️ ไม่มี open orders
💡 ลองส่ง limit order ใน Notebook 5 แล้วกลับมาเช็คที่นี่


---

## 📈 Cell 3: ดู Open Positions

In [12]:
# ==========================================
# Function Definition
# ==========================================

def get_open_positions(client):
    """
    ดึงและแสดง Open Positions
    
    Args:
        client: GRVT client
    
    Returns:
        positions: List of positions
    """
    print_section_header("📈 Open Positions")
    
    try:
        # ดึง positions
        positions = client.fetch_positions()
        
        # กรองเฉพาะที่มี position
        open_positions = [p for p in positions if safe_float(p.get('size', 0)) != 0]
        
        if not open_positions:
            print("ℹ️ ไม่มี open positions")
            print("💡 ถ้ามี open orders รอให้ถูก filled ก่อน หรือใช้ market order")
            return positions
        
        print(f"💼 มี {len(open_positions)} open position(s):\n")
        
        total_unrealized_pnl = 0
        
        for pos in open_positions:
            symbol = pos.get('instrument', 'N/A')
            size = safe_float(pos.get('size', 0))
            side = 'LONG 📈' if size > 0 else 'SHORT 📉'
            entry_price = safe_float(pos.get('entry_price', 0))
            mark_price = safe_float(pos.get('mark_price', 0))
            unrealized_pnl = safe_float(pos.get('unrealized_pnl', 0))
            leverage = safe_float(pos.get('leverage', 1))
            notional = safe_float(pos.get('notional', 0))
            roi = safe_float(pos.get('roi', 0))
            
            total_unrealized_pnl += unrealized_pnl
            
            print(f"  {symbol}")
            print(f"    Side:      {side}")
            print(f"    Size:      {abs(size)}")
            print(f"    Entry:     {format_currency(entry_price)}")
            print(f"    Mark:      {format_currency(mark_price)}")
            print(f"    Leverage:  {leverage}x")
            print(f"    Notional:  {format_currency(notional)}")
            print(f"    ROI:       {roi*100:.2f}%")
            print(f"    P&L:       {format_currency(unrealized_pnl)}")
            print()
        
        print(f"💰 Total Unrealized P&L: {format_currency(total_unrealized_pnl)}")
        return positions
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return []


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: ดู Open Positions"""
    positions = get_open_positions(client)
    return positions


# เรียกใช้
positions = main()


  📈 Open Positions

💼 มี 1 open position(s):

  BTC_USDT_Perp
    Side:      SHORT 📉
    Size:      0.002
    Entry:     $84,732.17
    Mark:      $82,958.72
    Leverage:  50.0x
    Notional:  -$165.92
    ROI:       1380.55%
    P&L:       $3.55

💰 Total Unrealized P&L: $3.55


---

## 📋 Cell 4: แสดง Positions ในรูปแบบตาราง

In [13]:
# ==========================================
# Function Definition
# ==========================================

def display_positions_table(positions):
    """
    แสดง Positions ในรูปแบบตาราง
    
    Args:
        positions: List of positions
    """
    try:
        positions_df = create_positions_table(positions)
        
        if not positions_df.empty:
            print("\n📋 Positions Table:\n")
            
            # จัดรูปแบบตาราง
            def color_pnl(val):
                """ระบายสีตาม P&L"""
                if isinstance(val, str) and '$' in val:
                    if val.startswith('-'):
                        return 'color: red; font-weight: bold'
                    else:
                        return 'color: green; font-weight: bold'
                return ''
            
            # แสดงตาราง
            styled_df = positions_df.style\
                .map(color_pnl, subset=['Unrealized P&L'])\
                .set_table_styles([{
                    'selector': 'th',
                    'props': [('background-color', '#2196F3'), ('color', 'white'), ('font-weight', 'bold')]
                }])\
                .set_properties(**{'text-align': 'left'})
            
            display(styled_df)
        else:
            print("\nℹ️ ไม่มี positions ที่แสดงในตาราง")
            
    except Exception as e:
        print(f"\n❌ Error creating table: {e}")
        import traceback
        traceback.print_exc()


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: แสดง Positions Table"""
    # ใช้ positions จาก cell ก่อนหน้า
    display_positions_table(positions)


# เรียกใช้
main()


📋 Positions Table:



,Symbol,Side,Size,Entry Price,Mark Price,Unrealized P&L
0,BTC_USDT_Perp,SHORT,0.002000,"$84,732.17","$82,958.72",$3.55


---

## 📊 Cell 5: สร้าง Balance Visualization

In [14]:
# ==========================================
# Function Definition
# ==========================================

def visualize_balance(balance):
    """
    สร้าง Balance Visualization
    
    Args:
        balance: Balance data
    """
    print_section_header("📊 Balance Visualization")
    
    try:
        # Extract data
        available = safe_float(balance.get('free', {}).get('USDT', 0))
        used = safe_float(balance.get('used', {}).get('USDT', 0))
        
        # สร้าง Pie chart
        fig = go.Figure(data=[go.Pie(
            labels=['Available', 'Used Margin'],
            values=[available, used],
            hole=0.4,
            marker=dict(colors=['#4CAF50', '#FF9800']),
            textinfo='label+percent+value',
            texttemplate='%{label}<br>$%{value:,.2f}<br>(%{percent})'
        )])
        
        fig.update_layout(
            title="Balance Distribution",
            showlegend=True,
            height=400
        )
        
        fig.show()
        
    except Exception as e:
        print(f"❌ Error creating chart: {e}")


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: สร้าง Balance Visualization"""
    # ใช้ balance จาก cell ก่อนหน้า
    visualize_balance(balance)


# เรียกใช้
main()


  📊 Balance Visualization



---

## 🔄 Cell 6: Real-time Monitoring (Auto-refresh)

⚠️ รัน cell นี้แล้วจะ **refresh ข้อมูลทุก 5 วินาที** (จำกัด 12 ครั้ง = 1 นาที)  
กด **Stop** (⬛) เพื่อหยุด

In [16]:
# ==========================================
# Function Definition
# ==========================================

def display_dashboard(client):
    """
    แสดง dashboard แบบ real-time
    
    Args:
        client: GRVT client
    """
    try:
        # ดึงข้อมูล
        balance = client.fetch_balance()
        positions = client.fetch_positions()
        open_orders = client.fetch_open_orders()
        
        # Extract metrics
        total_equity = safe_float(balance.get('total', {}).get('USDT', 0))
        available = safe_float(balance.get('free', {}).get('USDT', 0))
        used_margin = safe_float(balance.get('used', {}).get('USDT', 0))
        
        # Count open positions and orders
        open_pos = [p for p in positions if safe_float(p.get('size', 0)) != 0]
        total_pnl = sum(safe_float(p.get('unrealized_pnl', 0)) for p in open_pos)
        num_orders = len(open_orders)
        
        # Clear และแสดงผล
        clear_output(wait=True)
        
        print("=" * 60)
        print(f"  🎯 GRVT Account Dashboard  [{datetime.now().strftime('%H:%M:%S')}]")
        print("=" * 60)
        print()
        print(f"💎 Total Equity:        {format_currency(total_equity)}")
        print(f"✅ Available Balance:   {format_currency(available)}")
        print(f"📊 Used Margin:         {format_currency(used_margin)}")
        print(f"📝 Open Orders:         {num_orders}")
        print(f"📈 Open Positions:      {len(open_pos)}")
        print(f"💰 Unrealized P&L:      {format_currency(total_pnl)}")
        print()
        print("=" * 60)
        print("กด Stop (⬛) เพื่อหยุด auto-refresh")
        print("=" * 60)
        
    except Exception as e:
        print(f"❌ Error updating dashboard: {e}")


def auto_refresh_dashboard(client, refresh_interval=5, num_refreshes=12):
    """
    Auto-refresh dashboard
    
    Args:
        client: GRVT client
        refresh_interval: วินาที (default: 5)
        num_refreshes: จำนวนครั้ง (default: 12 = 1 นาที)
    """
    print("🔄 เริ่มต้น auto-refresh...\n")
    
    for i in range(num_refreshes):
        display_dashboard(client)
        time.sleep(refresh_interval)
    
    print("\n✅ Auto-refresh จบแล้ว")


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """
    Main: Auto-refresh Dashboard
    
    ⚠️ รัน cell นี้จะ refresh ทุก 5 วินาที (12 ครั้ง = 1 นาที)
    กด Stop (⬛) เพื่อหยุด
    """
    auto_refresh_dashboard(
        client,
        refresh_interval=5,  # ทุก 5 วินาที
        num_refreshes=12     # 12 ครั้ง = 1 นาที
    )


# เรียกใช้
main()

  🎯 GRVT Account Dashboard  [22:01:59]

💎 Total Equity:        $198,084.95
✅ Available Balance:   $197,241.07
📊 Used Margin:         $843.88
📝 Open Orders:         0
📈 Open Positions:      1
💰 Unrealized P&L:      -$9.20

กด Stop (⬛) เพื่อหยุด auto-refresh

✅ Auto-refresh จบแล้ว


---

## 📖 สรุป

### ✅ สิ่งที่เรียนรู้:

1. ✅ **Account Balance** - ดู Total Equity, Available, Used Margin
2. ✅ **Open Orders** - ดูคำสั่งที่รอ filled
3. ✅ **Open Positions** - ดูสินทรัพย์ที่ถือครองอยู่
4. ✅ **Unrealized P&L** - คำนวณกำไร/ขาดทุน
5. ✅ **Visualization** - แสดงผลในรูปแบบกราฟ
6. ✅ **Real-time Monitoring** - Auto-refresh ทุก 5 วินาที

### 📋 Functions ที่มี:

- `get_account_balance()` - ดึง Balance
- `display_balance_table()` - แสดง Balance เป็นตาราง
- `get_open_orders()` - ดึง Open Orders
- `get_open_positions()` - ดึง Open Positions
- `display_positions_table()` - แสดง Positions เป็นตาราง
- `visualize_balance()` - สร้างกราฟ Balance
- `display_dashboard()` - แสดง Real-time Dashboard
- `auto_refresh_dashboard()` - Auto-refresh Dashboard

### 💡 วิธีใช้งาน:

1. **รัน Cell 0** ก่อนเสมอ (Setup)
2. **รัน Cell 1-5** เพื่อดูข้อมูลแต่ละส่วน
3. **รัน Cell 6** เพื่อ Auto-refresh Real-time (ทุก 5 วินาที)
4. **กด Stop (⬛)** เพื่อหยุด Auto-refresh

### 🎯 ขั้นตอนต่อไป:

ไปที่ **Notebook 5: Order Execution** เพื่อ:
- ส่ง Limit Orders
- ส่ง Market Orders
- Cancel Orders
- จัดการ Positions

---

**Version:** Functions + main() + Auto-refresh 1.0